In [8]:
%load_ext autoreload
%autoreload 2
import pickle
import numpy as np
import tqdm
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from evaluate_numeral_context_lm import Evaluator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
res_path = '../numerate_language_models/experiments/wiki1B/checkpoints/b2_2020_01_17_05_51_24_wiki1B/log_test.txt.results.pkl'
results = pickle.load(open(res_path, 'rb'))
filtered_dataset = pickle.load(open('../../data/numeral_context/build/data_filtered_1Bnotable.test.rand.pkl','rb'))

In [3]:
all_num = [i[0] for i in filtered_dataset]
num2idx = {num:idx for idx, num in enumerate(all_num)}
all_score = np.zeros((len(all_num), len(all_num)))
all_score_adj = np.zeros((len(all_num), len(all_num)))

In [12]:
for res in tqdm.tqdm_notebook(results):
    ids =res[0]
    nlls = res[1]
    nll_adjs = res[2]
    
    assert len(ids) == len(nlls)
    assert len(ids) == len(nll_adjs)
    
    L = len(ids)
    
    for l in range(L):
    
        splits = ids[l].split(':')
        T_num = str(float(splits[1]))
        O_num = str(float(splits[3]))
        T_num_idx = num2idx[T_num]
        O_num_idx = num2idx[O_num]

        all_score[O_num_idx, T_num_idx] = -nlls[l]
        all_score_adj[O_num_idx, T_num_idx] = -nll_adjs[l]

In [13]:
e1 = Evaluator(filtered_dataset, all_score, all_score_adj)

In [14]:
e1.eval_all()

{'x': [70, 140, 280, 420, 700, 979],
 'topk': array([[0.05      , 0.1       , 0.2       , 0.3       , 0.5       ,
         0.69928571],
        [0.10214286, 0.19571429, 0.36857143, 0.48714286, 0.61642857,
         0.75928571],
        [0.10214286, 0.19571429, 0.36857143, 0.48714286, 0.61642857,
         0.75928571]]),
 'RMSE': array([1.86019223e+12, 1.86019223e+12]),
 'mape': array([97.67122956, 97.67122956]),
 'mae': array([1.72235284e+11, 1.72235284e+11]),
 'mdae': array([890., 890.]),
 'mdape': array([0.99365227, 0.99365227]),
 'avg_rank': [700.5, 571.3678571428571, 571.3678571428571]}